# Test Multi-Class Implementation

In [15]:
from gpkit import Model, Variable

### Some testing of model and variable inhertiance

In [16]:
class TestA(Model):
    def setup(self, extra=None):
        self.a = a = Variable('a', 5)
        self.b = b = Variable('b', 6)
        self.c = c = Variable('c')
        
        self.constr = self.get_constraints()        
        return self.constr
    
    def get_constraints(self):
        return [self.c >= self.a + self.b]

In [17]:
class TestB(Model):
    def setup(self, testa):
        testa.substitutions[testa.a] = 10  # try using substitutions --> Doesn't Work
        return testa  # need to return entire object in order to get the substitutions

In [39]:
class TestC(Model):
    def setup(self, testl):
        constr = []
        constr.extend(testl)  # this seems to work just returning the object as part of the subs
        return constr

In [40]:
testa = TestA()

In [41]:
testb = TestB(testa)

In [42]:
m = TestC(testb)

In [43]:
m.cost = testa.c

In [44]:
print(m.solve().table())

Using solver 'cvxopt'
Solving for 1 variables.
Solving took 0.00679 seconds.

Cost
----
 16

Free Variables
--------------
c : 16

Constants
---------
a : 10
b : 6

Sensitivities
-------------
a : +0.63
b : +0.38

Tightest Constraints
--------------------
    +1 : c >= a + b



## `MCell` Testing

In [13]:
from gpkit import units

In [14]:
from gpx.multiclass.mccell import MCell

In [15]:
from gpx.manufacturing import Process, QNACell

In [16]:
import numpy as np

In [17]:
cells = [QNACell(process=Process(), qnaconstr=False), 
         QNACell(process=Process(),qnaconstr=False)]

In [18]:
m = MCell(cells=cells)

In [19]:
# add the cells to the constraints
# m.extend(cells)

In [20]:
# add constraints to both cells to loop variations
m.extend([c.c2a >= c.c2d for c in cells])

In [21]:
# substitutions for cell rates
# where do these need to be integrated? at the MClass?
subs_to_update = {
    cells[0].lam : 10,
    cells[1].lam : 5,
    # cell cv (since not in processes)
    cells[0].c2s : 0.5,
    cells[1].c2s : 0.5,
    # process times
    cells[0].process.t : 10,
    cells[1].process.t : 5,
}

In [22]:
# add all cells substitutions
for c in cells:
    m.substitutions.update(c.substitutions)

In [23]:
m.substitutions.update(subs_to_update)

In [24]:
# m.substitutions[m.m] = 2

In [25]:
# cost function
# normally, WIP in the line is calculated at the fabline object
m.cost = np.sum([c.lam*c.W*units('count') for c in cells]) + m.m

In [26]:
sol = m.solve()
print(sol.table())

Using solver 'cvxopt'
Solving for 17 variables.
Solving took 0.101 seconds.

Cost
----
 5.927 [count]

Free Variables
--------------
              | MCell
    \bar{W_q} : 3.028  [min]      queueing time
 \bar{\alpha} : 0.325             1-rho
\bar{\lambda} : 15     [count/hr] aggregate production rate
   \bar{\rho} : 0.675             utilization
  \bar{c^2_a} : 0.54              Arrival coefficient of variation squared
  \bar{c^2_d} : 0.54              Departure coefficient of variation
  \bar{c^2_s} : 0.54              Service coefficient of variation
    \bar{t_s} : 8.333  [min]      service time
            m : 3.086  [count]    workstation count

              | QNACell6
            W : 13.03  [min]      Total flow time through cell
           Wq : 3.028  [min]      Expected queueing time
          c2a : 0.54              Arrival coefficient of variation squared
          c2d : 0.54              Departure coefficient of variation squared

              | QNACell7
            W : 8

## `MClass` Testing

In [1]:
from gpx.manufacturing import QNACell, Process, FabLine

In [2]:
from gpx.multiclass import MCSystem, MClass

In [3]:
import numpy as np

In [4]:
p1 = Process()
prod1 = MClass(
    line=FabLine([QNACell(process=p1) for _ in range(3)], return_cells=False),
    rate=10
)

In [5]:
p2 = Process()
prod2 = MClass(
    line=FabLine([QNACell(process=p2) for _ in range(3)], return_cells=False),
    rate=5
)

In [6]:
classes = [prod1, prod2]

In [7]:
# cellmap = [[p.cells[i] for p in classes] for i in range(3)]  # share all cells
cellmap = [[p.cells[i] for p in classes] for i in range(2)]  # share all but last cells

In [8]:
system = MCSystem(
    classes=classes,
    cellmap=cellmap
)

In [9]:
# this should be showing the other substitutions for cell rate and line rate
classes[0].substitutions

{QNACell.\lambda: 10,
 QNACell1.\lambda: 10,
 QNACell2.\lambda: 10,
 FabLine.\lambda: 10,
 MClass.\lambda: 10}

In [10]:
# subsitutions are not appearing in the system subsitutions
# but returns class substitutions

# system.substitutions[cellmap[0][0].lam]

In [11]:
# cost the WIP and the number of cells
system.cost = np.sum([c.line.L for c in classes])*0.1 + \
              np.sum([c.m for c in system.mcells]) + \
              np.sum([c.m for c in system.single_cells])

In [12]:
# substitute process times, cv
system.substitutions.update({
    prod1.cells[0].process.t : 5,
    prod1.cells[0].process.stdev : 5,
    prod2.cells[0].process.t : 10,
    prod2.cells[0].process.stdev : 5,
})

In [13]:
sol = system.solve()
print(sol.table())

Using solver 'cvxopt'
Solving for 62 variables.
Solving took 0.504 seconds.

Cost
----
 7.452 [count]

Free Variables
--------------
              | FabLine
            L : 8.079   [count]    Total WIP count
            W : 0.8079  [hr]       Total flow time

              | FabLine1
            L : 5.87    [count]    Total WIP count
            W : 1.174   [hr]       Total flow time

              | MCSystem.MCell
    \bar{W_q} : 11.14   [min]      queueing time
 \bar{\alpha} : 0.1601             1-rho
\bar{\lambda} : 15      [count/hr] aggregate production rate
   \bar{\rho} : 0.8399             utilization
  \bar{c^2_a} : 0.7008             Arrival coefficient of variation squared
  \bar{c^2_d} : 0.5846             Departure coefficient of variation
  \bar{c^2_s} : 0.5625             Service coefficient of variation
    \bar{t_s} : 6.667   [min]      service time
            m : 1.984   [count]    workstation count

              | MCSystem.MCell1
    \bar{W_q} : 10.6    [min]      